In [ ]:
!pip install pyabf==2.3.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 KB 31.9 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import cell_analysis_methods as methods
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def get_cells(date, data_folder = '/work/data'):
    cells = []
    for fn in os.listdir(os.path.join('/work/data', date)):
        start = fn.find('_') + 5
        stop = fn[start + 1:].find('_') + start + 1
        cell_n = int(fn[start:stop])
        if cell_n not in cells:
            cells.append(cell_n)
    return np.array(cells)

def split_IV_data(data):
    """
    split IV data into resting and holding
    """
    ordered_files = []
    prot_n = [int(fn[-8:-4]) for fn in data.keys()]
    rest = data[[fn for fn in data.keys() if fn.endswith("{:04d}.abf".format(min(prot_n)))][0]]
    holding = data[[fn for fn in data.keys() if fn.endswith("{:04d}.abf".format(max(prot_n)))][0]]

    ordered_files = [rest, holding]
    return ordered_files

def just_data(data):
    for data_file in data.keys():
        return data[data_file]
        
def run_analysis(data_path = '/work/data', save_path = '/work'):
    dates = os.listdir(data_path)
    all_cells = {'date': [],
                'cell': [],
                'RMP':[],
                'rheobase': [],
                'threshold': [],
                'height': [],
                'width': [],
                'spikes': [],
                'spikes_hold': [],
                'spike_n': [],
                'spike_n_hold':[],
                'spike_r':[],
                'spike_r_hold':[],
                'spike_adapt': [],
                'spike_adapt_hold': [],
                'Rm':[],
                'tau':[]
                }

    date_cell = {}
    total_cells = 0
    for date in dates:
        date_cell[date] = get_cells(date)
        total_cells += len(date_cell[date])

    print("total cells: {}".format(total_cells))
    with tqdm(total = total_cells) as progress:
        for date in tqdm(dates):
            cells = date_cell[date]
            for cell in cells:
                data_files = methods.get_data(date, cell)
                data_contained = data_files.keys()
            
                cell_RMP = methods.get_RMP(data_files)
                try:
                    cell_rheobase = methods.get_Rheobase(data_files)
                    cell_threshold = methods.get_spike_threshold(data_files)
                    cell_height = methods.get_spike_height(data_files)
                    cell_width = methods.get_spike_width(data_files)
                except:
                    cell_rheobase = np.nan
                    cell_threshold = np.nan
                    cell_height = np.nan
                    cell_width = np.nan
            
                try:
                    cell_IV_spikes = methods.get_IV_spikes(data_files)
                    cell_spikes = split_IV_data(cell_IV_spikes)
                    
                    cell_IV_spike_n = methods.get_IV_spike_number(data_files)
                    cell_spike_n = split_IV_data(cell_IV_spike_n)

                    cell_IV_spike_r = methods.get_IV_spike_rate(data_files)
                    cell_spike_r = split_IV_data(cell_IV_spike_r)

                    cell_IV_spike_adapt = methods.get_IV_adaptation(data_files)
                    cell_spike_adapt = split_IV_data(cell_IV_spike_adapt)

                except:
                    cell_spikes = [np.nan, np.nan]
                    cell_spike_n = [np.nan, np.nan]
                    cell_spike_r = [np.nan, np.nan]
                    cell_spike_adapt = [np.nan, np.nan]
                
                try:
                    cell_Rm = just_data(methods.get_Rm(data_files))
                except:
                    cell_Rm = np.nan
                
                try:
                    cell_tau = just_data(methods.get_tau(data_files))
                except:
                    cell_tau = np.nan

                all_cells['date'].append(date)
                all_cells['cell'].append(cell)
                all_cells['RMP'].append(cell_RMP)
                all_cells['rheobase'].append(cell_rheobase)
                all_cells['threshold'].append(cell_threshold)
                all_cells['height'].append(cell_height)
                all_cells['width'].append(cell_width)
                all_cells['spikes'].append(cell_spikes[0])
                all_cells['spikes_hold'].append(cell_spikes[1])
                all_cells['spike_n'].append(cell_spike_n[0])
                all_cells['spike_n_hold'].append(cell_spike_n[1])
                all_cells['spike_r'].append(cell_spike_r[0])
                all_cells['spike_r_hold'].append(cell_spike_r[1])
                all_cells['spike_adapt'].append(cell_spike_adapt[0])
                all_cells['spike_adapt_hold'].append(cell_spike_adapt[1])
                all_cells['Rm'].append(cell_Rm)
                all_cells['tau'].append(cell_tau)   

                progress.update(1)
        
    all_cells = pd.DataFrame(all_cells)
    file_path = '{}/all_cell_data'.format(save_path)
    all_cells.to_pickle(file_path)
    print('data saved as pickle: {}'.format(file_path))
    return all_cells

In [ ]:
run_analysis()
all_cells

total cells: 53
100%|██████████| 53/53 [01:56<00:00,  2.20s/it]
data saved as pickle: /work/all_cell_data


NameError: name 'all_cells' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=99013d08-680f-4983-9780-1a2087bba930' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>